<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2021/blob/main/Pandas_Analyze_Maintenance_Log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/profteachkids/CHE5136_Fall2021/main/maintenance_log.txt

--2021-08-28 18:27:48--  https://raw.githubusercontent.com/profteachkids/CHE5136_Fall2021/main/maintenance_log.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26025250 (25M) [text/plain]
Saving to: ‘maintenance_log.txt’

maintenance_log.txt 100%[===================>]  24.82M   141MB/s    in 0.2s    

2021-08-28 18:27:50 (141 MB/s) - ‘maintenance_log.txt’ saved [26025250/26025250]



In [50]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('/content/maintenance_log.txt')
df.head()

,start,end,equipment,task,name,cost
0,2018-10-26,2018-11-09,ReactorA,Overhaul,Corrine,791158.32
1,2018-02-19,2018-03-03,TankE,Inspect,Corrine,88.50
2,2020-04-05,2020-04-09,HeaterC,Inspect,Garek,42.78
3,2019-07-15,2019-07-24,ReactorD,Overhaul,George,570538.03
4,2020-07-29,2020-08-02,ReactorE,Overhaul,Corrine,925844.80


In [4]:
df=df.astype({'equipment':'category','name':'category','cost':'float64','start':'datetime64[ns]', 'end':'datetime64[ns]'})
df.dtypes


start        datetime64[ns]
end          datetime64[ns]
equipment          category
task                 object
name               category
cost                float64
dtype: object

In [5]:
df['days']=(df['end']-df['start']).dt.days

In [7]:
df2=df.set_index(['start','equipment','task','name'])
df2.sort_index(inplace=True)
idx = pd.IndexSlice

In [ ]:
df2.loc[idx['2020-05-30':'2020-06-5',['ColumnA','TankA'],'Inspect',:]]

In [9]:
df3=pd.pivot_table(data=df,values=['cost','days'],columns=['equipment','task','name'],aggfunc=['mean','min','max'])

In [10]:
df3

mean                ...       max          
equipment       ColumnA                ...     TankE          
task              Clean                ...  Overhaul          
name             Carlos       Corrine  ...     Garek    George
cost       60144.905952  32009.036082  ...  14033.56  18094.94
days          15.264848     14.986998  ...     29.00     29.00

[2 rows x 900 columns]

In [11]:
df4=df3.stack([1,2,3])

In [12]:
df5=df4[['mean','max','min']]
df5['mean']=df5['mean'].round(2)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(df5.loc[idx['cost']])

In [ ]:
def f(x):
    return x.startswith('Column')

df5[df5.index.get_level_values('equipment').map(f)].loc[idx['cost']]

In [15]:
df_Mondays=df2[df2.index.get_level_values('start').dayofweek==0]

In [16]:
df_Monday_pivot=pd.pivot_table(data=df_Mondays,values=['cost'],columns=['equipment','task','name'],aggfunc=['mean'])

In [17]:
df_Monday_pivot.stack([1,2,3])

mean
     equipment task     name                 
cost ColumnA   Clean    Carlos   60781.905240
                        Corrine  32140.053096
                        Garek    36110.569150
                        George   44223.958755
               Inspect  Carlos    6040.615880
...                                       ...
     TankE     Inspect  George     132.085153
               Overhaul Carlos   18083.666206
                        Corrine   9652.829244
                        Garek    10820.031441
                        George   13157.748299

[300 rows x 1 columns]

In [20]:
dates=pd.date_range(start='28 Aug 2019', end='28 Aug 2021')

In [26]:
intervals = pd.arrays.IntervalArray.from_tuples(list(zip(df['start'],df['end'])))

In [57]:
for date in dates[:3]:
    print(df.loc[intervals.contains(date)])


            start        end  ... days             date_interval
35     2019-08-24 2019-09-17  ...   24  (2019-08-24, 2019-09-17]
68     2019-08-24 2019-09-11  ...   18  (2019-08-24, 2019-09-11]
177    2019-08-24 2019-09-15  ...   22  (2019-08-24, 2019-09-15]
262    2019-08-18 2019-09-10  ...   23  (2019-08-18, 2019-09-10]
354    2019-08-23 2019-09-05  ...   13  (2019-08-23, 2019-09-05]
...           ...        ...  ...  ...                       ...
499163 2019-08-14 2019-09-01  ...   18  (2019-08-14, 2019-09-01]
499171 2019-08-27 2019-09-13  ...   17  (2019-08-27, 2019-09-13]
499177 2019-08-24 2019-09-13  ...   20  (2019-08-24, 2019-09-13]
499722 2019-08-27 2019-08-30  ...    3  (2019-08-27, 2019-08-30]
499740 2019-08-27 2019-09-21  ...   25  (2019-08-27, 2019-09-21]

[4060 rows x 8 columns]
            start        end  ... days             date_interval
35     2019-08-24 2019-09-17  ...   24  (2019-08-24, 2019-09-17]
68     2019-08-24 2019-09-11  ...   18  (2019-08-24, 2019-09-11]
